# Mobility and Storage Time Calculator

The following code takes processed and "cleaned" water masks from a specified working directory and performs four different operations to calculate the area-based channel mobility and the sediment storage time distributions of river reaches. The first operation is conducted by running the "get_mobility_dfs()" function which produces dataframes for each reach of the given river, calculating the evolution of wet and dry pixels over time; these dataframes are output as .csvs to local folders. The second operation is conducted by running the get_TR_and_AW_distribution()" which calculates the lateral channel mobility of each reach (TR) and outputs a .csv with TR by reach, as well as individual .csvs for each reach including all AW values (one from each annual median water mask). The third opertion is conducted by running the get_TCB() function, and calculates a distribution of channel belt turnover times (TCB), outputting them to reach-specific .csvs. The fourth and final operation is conducted by running the get_TFP(), which runs the random walk model and develops a first passage time distriubtion (storage time distribution) that is output as a .csv.

Author: James (Huck) Rees; PhD Student, UCSB Geography

Date: January 17, 2025

## Import packages

In [1]:
import os
import numpy as np
import pandas as pd
from natsort import natsorted
from glob import glob
import math
import geopandas as gpd
import ast

import re
import fiona
import rasterio
from rasterio.mask import mask
from rasterio import warp
from rasterio.warp import transform_geom
from rasterio.enums import Resampling
from pyproj import CRS

from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter

import geemap
import ee
from geopy.distance import geodesic

# Authenticate with Google Earth Engine
ee.Initialize()

## Initialize functions to produce mobility dataframes and CSVs

In [ ]:
def get_mobility_yearly(images, mask, scale=30):

    A = len(np.where(mask == 1)[1])

    year_range = list(images.keys())
    ranges = [year_range[i:] for i, yr in enumerate(year_range)]
    river_dfs = {}
    for yrange in ranges:
        data = {
            'year': [],
            'i': [],
            'O_avg': [],
            'O_wd': [],
            'O_dw': [],
            'O_wick': [],
            'fR': [],
            'fR_wick': [],
            'w_b': [],
            'd_b': [],
        }
        length = images[yrange[0]].shape[0]
        width = images[yrange[0]].shape[1]
        long = len(yrange)
        all_images = np.empty((length, width, long))
        years = []
        for j, year in enumerate(yrange):
            years.append(year)
            im = images[str(year)].astype(int)
            filt = np.where(~np.array(mask) + 2)
            im[filt] = 0
            all_images[:, :, j] = im

        baseline = all_images[:, :, 0]
        w_b = len(np.where(baseline == 1)[0])
        fb = mask - baseline
        fw_b = w_b / A
        fd_b = np.sum(fb) / A
        Na = A * fd_b

        for j in range(all_images.shape[2]):
            im = all_images[:, :, j]

            kb = (
                np.sum(all_images[:, :, :j + 1], axis=(2))
                + mask
            )
            kb[np.where(kb != 1)] = 0
            Nb = np.sum(kb)
            # fR = (Na / w_b) - (Nb / w_b)
            fR = (Na - Nb)
            fR_wick = 1 - (Nb / Na)

            # Calculate D - EQ. (1)
            D = np.subtract(baseline, im)
            # 1 - wet -> dry
            d_wd = len((np.where(D == 1))[0])
            # -1 - dry -> wet
            d_dw = len((np.where(D == -1))[0])

            # Calculate Phi
            w_t = len(np.where(im == 1)[0])
            fw_t = w_t / A
            fd_t = (A - w_t) / A

            # Calculate O_Phi
            PHI = (fw_b * fd_t) + (fd_b * fw_t)
            o_wick = 1 - (np.sum(np.abs(D)) / (A * PHI))
            o_avg = w_b - np.mean([d_wd, d_dw])
            o_wd = w_b - d_wd
            o_dw = w_b - d_dw

            data['i'].append(j)
            data['O_avg'].append(o_avg * (scale**2))
            data['O_wd'].append(o_wd * (scale**2))
            data['O_dw'].append(o_dw * (scale**2))
            data['O_wick'].append(o_wick)
            data['fR'].append(fR * (scale**2))
            data['fR_wick'].append(fR_wick)
            data['w_b'].append(w_b * (scale**2))
            data['d_b'].append(Na * (scale**2))

        data['year'] = years
        data['i'] = np.array(years).astype(int) - int(years[0])
        river_dfs[yrange[0]] = pd.DataFrame(data=data)

    return river_dfs

def get_mobility_rivers(folder_path, river, mob_storage, reach_range="All"):
    """
    Processes mobility metrics for specified reaches of a river and saves
    the results to separate CSV files with the reach number in the filename.

    Parameters:
        folder_path (str): The root directory that contains subfolders for each reach.
        river (str): Name of the river.
        mob_storage (str): Path to store the output CSV files.
        reach_range (str/int/tuple): Specifies which reaches to process. Can be:
            - An integer for a single reach (e.g., 3).
            - A tuple for a range of reaches (e.g., (1, 4)).
            - "All" to process all reaches.

    Returns:
        str: Name of the river (for confirmation or chaining).
    """
    import os
    from glob import glob
    import numpy as np
    import pandas as pd
    from natsort import natsorted

    def create_mask_shape(river, fps):
        """
        Reads the first raster file in fps and applies a mask processing step to return a binary raster
        where pixel values are transformed as follows:
            - Values < 10 are set to 0
            - Values >= 10 are set to 1

        Parameters:
        river (str): Unused in this version, kept for consistency with the original function signature.
        fps (list): List of file paths to raster images, with the first path being used for processing.

        Returns:
        numpy.ndarray: A 2D array with the processed binary mask.
        """
        import rasterio

        # Use the first file path in fps to open the raster file
        image = fps[0]
        with rasterio.open(image) as ds:
            # Read the data as a 2D array (assuming single-band raster)
            out_image = ds.read(1).astype('int64')
            
            # Mask processing
            out_image += 11  # Offset values as in the original function
            out_image[np.where(out_image < 10)] = 0
            out_image[np.where(out_image > 10)] = 1

        return out_image

    def clean(river, fps):
        """
        Processes a set of raster images by creating binary water masks without using any polygon for masking.
        Each raster file in `fps` is read, transformed into a binary water mask, and stored in a dictionary
        by year (derived from filenames).

        Parameters:
        river (str): Unused in this version, kept for compatibility with the original function signature.
        fps (list): List of file paths to raster images.

        Returns:
        tuple: A dictionary of images (binary masks by year) and their respective metadata.
        """
        import rasterio
        from rasterio.enums import Resampling
        import re

        images = {}
        metas = {}

        # Process each file path in fps
        for fp in fps:
            # Extract year from filename using regex
            year_match = re.findall(r"[0-9]{4,7}", fp)
            if year_match:
                year = year_match[-1]  # Take the last match as the year
            else:
                continue  # Skip files without a year identifier

            # Open the raster file
            with rasterio.open(fp) as ds:
                # Read the data and apply threshold to create a binary water mask
                image = ds.read(1, resampling=Resampling.nearest) > 0  # Binary mask where pixel > 0 is water

                # Skip images with no water (all values are 0)
                if not np.any(image):
                    continue

                # Update metadata for the binary water mask
                meta = ds.meta
                meta.update(
                    width=image.shape[1],
                    height=image.shape[0],
                    count=1,
                    dtype=rasterio.int8
                )

                # Save the binary water mask and its metadata
                images[year] = image
                metas[year] = meta

        return images, metas

    # Ensure the storage directory exists
    if not os.path.exists(mob_storage):
        os.makedirs(mob_storage)

    print(f"Processing river: {river}")

    # Generate the dictionary of paths for each reach
    paths = {}
    river_folder = os.path.join(folder_path, river)
    for reach_folder in os.listdir(river_folder):
        reach_path = os.path.join(river_folder, reach_folder, 'Cleaned')
        if os.path.isdir(reach_path):
            tif_files = glob(os.path.join(reach_path, "*.tif"))
            if tif_files:
                paths[reach_folder] = tif_files

    # Filter reaches based on reach_range
    if isinstance(reach_range, int):
        # Single reach
        reach_keys = [f"reach_{reach_range}"]
    elif isinstance(reach_range, tuple):
        # Range of reaches
        start, end = reach_range
        reach_keys = [f"reach_{i}" for i in range(start, end + 1)]
    elif reach_range == "All":
        # All reaches
        reach_keys = list(paths.keys())
    else:
        raise ValueError("Invalid reach_range format. Must be an integer, tuple, or 'All'.")

    # Iterate through specified reaches
    for reach_key in reach_keys:
        if reach_key not in paths:
            print(f"Reach {reach_key} not found in paths. Skipping.")
            continue

        path_list = paths[reach_key]

        # Extract the reach number from the reach_key (assumes format "reach_x")
        reach_number = reach_key.split('_')[1]  # Gets the 'x' part of "reach_x"

        # Sort paths naturally
        path_list = natsorted(path_list)

        # Generate the mask for the reach
        mask = create_mask_shape(river, path_list)

        # Clean and retrieve images and metadata
        images, metas = clean(river, path_list)

        # Set a fixed scale for processing
        scale = 30

        # Calculate yearly mobility metrics
        river_dfs = get_mobility_yearly(images, mask, scale=scale)

        # Combine data into a full DataFrame for the reach
        full_df = pd.DataFrame()
        for year, df in river_dfs.items():
            rnge = f"{year}_{df.iloc[-1]['year']}"
            df['dt'] = pd.to_datetime(df['year'], format='%Y')
            df['range'] = rnge

            # Append data to the final DataFrame
            full_df = pd.concat([full_df, df], ignore_index=True)

        # Define the output path with reach number in the filename
        out_path = os.path.join(mob_storage, f'{river}_reach_{reach_number}_yearly_mobility.csv')
        full_df.to_csv(out_path, index=False)
        print(f"Saved mobility metrics for {river} reach {reach_number} to {out_path}")

def get_mobility_dfs(csv_path):
    """
    Wrapper function to process multiple rivers and reaches based on a CSV file.

    Parameters:
        csv_path (str): Path to the CSV file containing river and reach information.

    Returns:
        None
    """

    # Read the CSV file into a DataFrame
    river_data = pd.read_csv(csv_path)

    # Iterate over each row in the DataFrame
    for index, row in river_data.iterrows():
        river_name = row['river_name']
        working_directory = row['working_directory']
        reach_range = row['reach_range']

        # Parse reach_range
        if isinstance(reach_range, str) and reach_range != "All":
            if reach_range.startswith("(") and reach_range.endswith(")"):
                # Convert tuple-like string to actual tuple
                reach_range = tuple(map(int, reach_range.strip("() ").split(",")))
            else:
                # Single integer as string
                reach_range = int(reach_range)

        # Construct input and output paths
        folder_path = f"{working_directory}/RiverMapping/RiverMasks"
        mob_storage = os.path.join(working_directory, "RiverMapping", "Mobility", river_name, "Mobility_dfs")
        os.makedirs(mob_storage, exist_ok=True)

        # Call the get_mobility_dfs function for the current river
        print(f"Processing {river_name} with reach range {reach_range}...")
        get_mobility_rivers(folder_path, river_name, mob_storage, reach_range)

    print("All rivers processed.")

## Initialize functions to calculate floodplain reworking timescale (TR) from mobility sheets

In [ ]:
def get_TR(csv_path):
    """
    Processes TR values for reaches based on a CSV file and generates aw distributions.

    Parameters:
        csv_path (str): Path to the CSV file containing river and reach information.

    Returns:
        None
    """
    import pandas as pd
    import os
    import numpy as np
    from scipy.optimize import curve_fit

    def calculate_median_fit_with_TR_and_uncertainty(data):
        """
        A helper function that computes TR from a DataFrame containing channel mobility data.
        """
        data['AR_over_AW'] = data['fR'] / data['w_b']
        grouped = data.groupby('i')
        median_values = []
        i_values = []
        for i, group in grouped:
            i_values.append(i)
            median_values.append(np.median(group['AR_over_AW']))

        i_values = np.array(i_values)
        median_values = np.array(median_values)

        def exp_decay_asymptote(i, PR_over_AW, CR):
            return -PR_over_AW * np.exp(-CR * i) + PR_over_AW

        initial_guess = [1, 0.1]
        params, _ = curve_fit(exp_decay_asymptote, i_values, median_values, p0=initial_guess, maxfev=10000)
        PR_over_AW, CR = params
        Aw = 1
        return (1 / CR) * (Aw / PR_over_AW)

    def get_aw_dist(input_directory, output_directory, reach_range):
        """
        Extracts the distribution of `w_b` values when `i=0` for specified reaches and saves them.

        Parameters:
            input_directory (str): Directory containing the mobility CSV files.
            output_directory (str): Directory to save the output CSV files.
            reach_range (str/int/tuple): Specifies which reaches to process. Can be:
                - An integer for a single reach (e.g., 3).
                - A tuple for a range of reaches (e.g., (1, 4)).
                - "All" to process all reaches.

        Returns:
            None
        """
        os.makedirs(output_directory, exist_ok=True)

        # List all mobility CSV files
        csv_files = [f for f in os.listdir(input_directory) if f.endswith("_yearly_mobility.csv")]

        for csv_file in csv_files:
            try:
                # Extract reach number
                reach_number = int(csv_file.split('_')[2])

                # Filter based on reach_range
                if isinstance(reach_range, int) and reach_number != reach_range:
                    continue
                elif isinstance(reach_range, tuple) and not (reach_range[0] <= reach_number <= reach_range[1]):
                    continue

                # Read the mobility CSV file
                data = pd.read_csv(os.path.join(input_directory, csv_file))

                # Filter rows where `i == 0` and get `w_b` values
                w_b_values = data.loc[data['i'] == 0, 'w_b'].tolist()

                # Create a DataFrame for the distribution
                output_df = pd.DataFrame(w_b_values, columns=['a_w'])

                # Save to a CSV file
                output_file = os.path.join(output_directory, f"Reach_{reach_number}_aw_dist.csv")
                output_df.to_csv(output_file, index=False)

                print(f"Saved `a_w` distribution for Reach {reach_number} to {output_file}")

            except Exception as e:
                print(f"Error processing {csv_file}: {e}")

    river_data = pd.read_csv(csv_path)
    for index, row in river_data.iterrows():
        river_name = row['river_name']
        working_directory = row['working_directory']
        reach_range = row['reach_range']

        # Parse reach_range
        if isinstance(reach_range, str) and reach_range != "All":
            if reach_range.startswith("(") and reach_range.endswith(")"):
                reach_range = tuple(map(int, reach_range.strip("() ").split(",")))
            else:
                reach_range = int(reach_range)

        input_directory = f"{working_directory}/RiverMapping/Mobility/{river_name}/Mobility_DFs"
        tr_output_directory = f"{working_directory}/RiverMapping/Mobility/{river_name}"
        aw_output_directory = f"{working_directory}/RiverMapping/Mobility/{river_name}/AW_Distributions"

        os.makedirs(tr_output_directory, exist_ok=True)
        os.makedirs(aw_output_directory, exist_ok=True)

        csv_files = [f for f in os.listdir(input_directory) if f.endswith("_yearly_mobility.csv")]
        results = []

        for csv_file in csv_files:
            reach_number = int(csv_file.split('_')[2])

            # Filter based on reach_range
            if isinstance(reach_range, int) and reach_number != reach_range:
                continue
            elif isinstance(reach_range, tuple) and not (reach_range[0] <= reach_number <= reach_range[1]):
                continue

            data = pd.read_csv(os.path.join(input_directory, csv_file))
            try:
                TR = calculate_median_fit_with_TR_and_uncertainty(data)
                results.append((reach_number, TR))
            except Exception as e:
                print(f"Error processing {csv_file}: {e}")

        results_df = pd.DataFrame(results, columns=['Reach_Number', 'TR'])
        results_df.sort_values(by='Reach_Number', inplace=True)

        output_file = os.path.join(tr_output_directory, f"{river_name}_TR_values.csv")
        results_df.to_csv(output_file, index=False)
        print(f"TR values saved to {output_file}")

        # Generate AW distributions for the specified reaches
        get_aw_dist(input_directory, aw_output_directory, reach_range)


## Initialize functions to calculate channel-belt turnover timescale (TCB) 

In [ ]:
def import_aw_distribution(river_name, reach_number, working_directory):
    """
    Imports the AW distribution for a specified reach.

    Args:
        river_name (str): Name of the river.
        reach_number (int): Reach number to import AW distribution.
        working_directory (str): Base working directory containing the river data.

    Returns:
        DataFrame: A DataFrame containing the AW distribution for the specified reach.
    """
    # Define base directory for AW distributions
    aw_dir = os.path.join(working_directory, 'RiverMapping', 'Mobility', river_name, 'AW_Distributions')

    # Ensure the directory exists
    if not os.path.exists(aw_dir):
        raise FileNotFoundError(f"AW distribution directory not found: {aw_dir}")

    # Load AW distribution for the specified reach
    aw_file = os.path.join(aw_dir, f"Reach_{reach_number}_aw_dist.csv")
    if not os.path.isfile(aw_file):
        raise FileNotFoundError(f"AW file not found: {aw_file}")

    aw_distribution = pd.read_csv(aw_file)

    return aw_distribution

def calculate_tcb_distribution(tr_value, channel_belt_area, aw_distribution):
    """
    Calculates the TCB distribution for a single reach and returns the result.

    Args:
        tr_value (float): TR value for the reach.
        channel_belt_area (float): Channel belt area (in square km) for the reach.
        aw_distribution (DataFrame): AW distribution for the reach.

    Returns:
        DataFrame: A DataFrame containing the TCB distribution for the specified reach.
    """
    # Ensure the AW distribution is not empty
    if aw_distribution.empty:
        raise ValueError("AW distribution is empty.")

    # Generate 10,000 random draws from the AW distribution
    aw_random_draws = np.random.choice(aw_distribution['a_w'], size=10000, replace=True)

    # Calculate TCB for each random draw, converting channel belt area from square km to square m
    tcb_values = tr_value * (channel_belt_area * 1000000 / aw_random_draws)

    return pd.DataFrame({'TCB': tcb_values})

def get_tcb_distributions(csv_path):
    """
    Processes a range of reaches from a CSV file and calculates TCB distributions for each.

    Args:
        csv_path (str): Path to the CSV file containing river name and reach range.

    Outputs:
        CSV files containing TCB distributions for each processed reach.
    """
    # Load the configuration CSV
    config_data = pd.read_csv(csv_path)

    # Extract river name, reach range, and working directory
    river_name = config_data['river_name'].iloc[0]
    reach_range = config_data['reach_range'].iloc[0]
    working_directory = config_data['working_directory'].iloc[0]

    # Define directories for required inputs
    tr_file = os.path.join(working_directory, 'RiverMapping', 'Mobility', river_name, f"{river_name}_TR_values.csv")
    channel_belt_file = os.path.join(working_directory, 'ChannelBelts', 'Extracted_ChannelBelts', river_name, f"{river_name}_channelbelt_areas.csv")

    # Check if the required files exist
    if not os.path.isfile(tr_file):
        raise FileNotFoundError(f"TR file not found: {tr_file}")
    if not os.path.isfile(channel_belt_file):
        raise FileNotFoundError(f"Channel belt areas file not found: {channel_belt_file}")

    # Load the TR and channel belt areas data
    tr_data = pd.read_csv(tr_file)
    channel_belt_data = pd.read_csv(channel_belt_file)

    # Parse reach_range
    if isinstance(reach_range, str) and reach_range != "All":
        if reach_range.startswith("(") and reach_range.endswith(")"):
            # Convert tuple-like string to actual tuple
            reach_range = tuple(map(int, reach_range.strip("() ").split(",")))
        else:
            # Single integer as string
            reach_range = reach_range.astype(int)
    
    # Determine the range of reaches to process
    if reach_range == "All":
        reaches = tr_data['Reach_Number'].unique()
    elif isinstance(reach_range, tuple):
        reaches = range(reach_range[0], reach_range[1] + 1)
    else:
        reaches = [reach_range]

    # Iterate through the range of reaches and calculate TCB for each
    for reach_number in reaches:
        # Get TR value for the reach
        tr_value = tr_data.loc[tr_data['Reach_Number'] == reach_number, 'TR'].values[0]

        # Get channel belt area for the reach
        channel_belt_area = channel_belt_data.loc[channel_belt_data['ds_order'] == reach_number, 'area_sq_km'].values[0]

        # Import AW distribution for the reach
        aw_distribution = import_aw_distribution(river_name, reach_number, working_directory)

        # Calculate the TCB distribution for the reach
        tcb_distribution = calculate_tcb_distribution(tr_value, channel_belt_area, aw_distribution)

        # Save TCB distribution to a CSV
        output_file = os.path.join(working_directory, 'RiverMapping', 'Mobility', river_name, 'TCB_Distributions', f"Reach_{reach_number}_TCB_distribution.csv")
        os.makedirs(os.path.dirname(output_file), exist_ok=True)
        tcb_distribution.to_csv(output_file, index=False)

        print(f"TCB distribution for Reach {reach_number} saved to {output_file}")

## Initialize functions to calculate first-passage time distributions of sediment storage time (TFP)

In [ ]:
def calculate_tfp_distribution(tr_value, channel_belt_area, aw_distribution):
    """
    Calculates the TFP distribution for a single reach using the random walk model and returns the result.

    Args:
        tr_value (float): TR value for the reach.
        channel_belt_area (float): Channel belt area (in square km) for the reach.
        aw_distribution (DataFrame): AW distribution (in square m) for the reach.

    Returns:
        DataFrame: A DataFrame containing the TFP distribution for the specified reach.
    """
    # Convert channel belt area from square km to square m
    channel_belt_area_m2 = channel_belt_area * 1000000

    # Ensure the AW distribution is not empty
    if aw_distribution.empty:
        raise ValueError("AW distribution is empty.")

    # Extract the 'a_w' values from the distribution
    aw_values = aw_distribution['a_w'].values

    # Number of iterations for the Monte Carlo simulation
    num_iterations = 10000

    # Maximum number of timesteps per iteration
    max_timesteps = 10000

    # Initialize storage for the first passage times
    tfp_times = []

    for _ in range(num_iterations):
        # Randomly initialize the channel's starting position within the domain
        x0 = np.random.uniform(0, channel_belt_area_m2)
        x = x0

        total_time = 0
        timestep_count = 0

        while timestep_count < max_timesteps:
            # Draw a random step size from the AW distribution
            aw_step = np.random.choice(aw_values)

            # Randomly determine the direction (+ or -)
            direction = np.random.choice([-1, 1])

            # Move the channel
            x_new = x + direction * aw_step

            # Reflect at boundaries if needed
            if x_new < 0:
                x_new = -x_new
            elif x_new > channel_belt_area_m2:
                x_new = 2 * channel_belt_area_m2 - x_new

            # Check if the channel has reached or passed the starting position
            if (x_new >= x0 and x < x0) or (x_new <= x0 and x > x0):
                # Calculate fractional time for the overshoot
                remaining_distance = abs(x0 - x)
                fractional_tr = (remaining_distance / aw_step) * tr_value
                total_time += fractional_tr
                break

            # Increment total time for this full step
            total_time += tr_value
            timestep_count += 1
            x = x_new

        # If the loop exited due to timestep limit, skip this iteration
        if timestep_count >= max_timesteps:
            continue

        # Store the total time for this iteration
        tfp_times.append(total_time)

    # Return the TFP distribution as a DataFrame
    return pd.DataFrame({'TFP': tfp_times})

def get_tfp_distributions(csv_path):
    """
    Processes a range of reaches from a CSV file and calculates TFP distributions for each.

    Args:
        csv_path (str): Path to the CSV file containing river name and reach range.

    Outputs:
        CSV files containing TFP distributions for each processed reach.
    """
    # Load the configuration CSV
    config_data = pd.read_csv(csv_path)

    for index, row in config_data.iterrows():
        # Extract river name, reach range, and working directory for each row
        river_name = row['river_name']
        reach_range = row['reach_range']
        working_directory = row['working_directory']

        # Define directories for required inputs
        tr_file = os.path.join(working_directory, 'RiverMapping', 'Mobility', river_name, f"{river_name}_TR_values.csv")
        channel_belt_file = os.path.join(working_directory, 'ChannelBelts', 'Extracted_ChannelBelts', river_name, f"{river_name}_channelbelt_areas.csv")

        # Check if the required files exist
        if not os.path.isfile(tr_file):
            raise FileNotFoundError(f"TR file not found: {tr_file}")
        if not os.path.isfile(channel_belt_file):
            raise FileNotFoundError(f"Channel belt areas file not found: {channel_belt_file}")

        # Load the TR and channel belt areas data
        tr_data = pd.read_csv(tr_file)
        channel_belt_data = pd.read_csv(channel_belt_file)

        # Determine the reach range
        if isinstance(reach_range, str):
            reach_range = reach_range.strip()  # Remove any extra spaces

            if reach_range == "All":
                reach_start = tr_data['Reach_Number'].min()
                reach_end = tr_data['Reach_Number'].max()
            elif reach_range.isdigit():
                # Convert a numeric string to an integer
                reach_range = int(reach_range)
                reach_start = reach_range
                reach_end = reach_range
            elif re.match(r'^\(\d{1,4}, \d{1,4}\)$', reach_range):  # Match (XX, YY) with 1 to 4 digits
                try:
                    # Convert the string to a tuple of integers
                    reach_range = ast.literal_eval(reach_range)
                    reach_start, reach_end = reach_range
                except (ValueError, SyntaxError):
                    raise ValueError(f"Invalid reach range format: {reach_range}")
            else:
                raise ValueError(f"Invalid string format for reach_range: {reach_range}")
        elif isinstance(reach_range, (int, float)) and float(reach_range).is_integer():
            # Convert float-like integers (e.g., 7.0) to int
            reach_range = int(reach_range)
            reach_start = reach_range
            reach_end = reach_range
        elif isinstance(reach_range, tuple) and len(reach_range) == 2:
            reach_start, reach_end = reach_range
        else:
            raise ValueError("reach_range must be 'All', an int, or a tuple (start, end).")

        # Generate range of reaches to process
        reaches = range(reach_start, reach_end + 1)

        # Iterate through the range of reaches and calculate TFP for each
        for reach_number in reaches:
            # Get TR value for the reach
            tr_value = tr_data.loc[tr_data['Reach_Number'] == reach_number, 'TR'].values[0]

            # Get channel belt area for the reach
            channel_belt_area = channel_belt_data.loc[channel_belt_data['ds_order'] == reach_number, 'area_sq_km'].values[0]

            # Import AW distribution for the reach
            aw_distribution = import_aw_distribution(river_name, reach_number, working_directory)

            # Calculate the TFP distribution for the reach
            tfp_distribution = calculate_tfp_distribution(tr_value, channel_belt_area, aw_distribution)

            # Save TFP distribution to a CSV
            output_file = os.path.join(working_directory, 'RiverMapping', 'Mobility', river_name, 'TFP_Distributions', f"Reach_{reach_number}_TFP_distribution.csv")
            os.makedirs(os.path.dirname(output_file), exist_ok=True)
            tfp_distribution.to_csv(output_file, index=False)

            print(f"TFP distribution for Reach {reach_number} saved to {output_file}")


In [ ]:
csv_path = r"C:\Users\huckr\Desktop\UCSB\Dissertation\Data\RiverMapping\Bermejo_river_datasheet.csv"

In [ ]:
get_mobility_dfs(csv_path)

In [ ]:
get_TR(csv_path)

In [ ]:
get_tcb_distributions(csv_path)

In [ ]:
get_tfp_distributions(csv_path)

## Initialize functions to calculate sediment transit times and velocities

In [ ]:
def import_wbsed(working_directory, river_name):
    """
    Imports a spreadsheet of sediment flux data from a specified local folder.

    Parameters:
        working_directory (str): The root directory containing the sediment flux data.
        river_name (str): The name of the target river.

    Returns:
        pd.DataFrame: A DataFrame containing the sediment flux data.
    """
    # Construct the full path to the file
    file_path = os.path.join(working_directory, 'WBMsed', f'{river_name}_wbmsed.csv')
    
    # Read the CSV file into a DataFrame
    return pd.read_csv(file_path)

In [ ]:
working_directory = r'D:\Dissertation\Data'
river_name = 'Beni'
beni = import_wbsed(working_directory, river_name)

In [ ]:
print(beni)

In [ ]:
def calculate_sed_flux(sed_flux_df, qs_inclusion = 1, sediment_bulk_density = 1600):
    """
    Calculates sediment flux by reach using different sediment flux combinations.
    
    Parameters:
        sed_flux_df (pd.DataFrame): DataFrame of sediment flux data from the WBMsed model.
        qs_inclusion (int): Determines which sediment flux combination to use (1 or 2).
            1 -> Bedload + Suspended Bedload
            2 -> Bedload + Suspended Bedload + Washload
        sediment_bulk_density (float): Estimated density of floodplain sediment in kg per cubic meter.

    Returns:
        pd.DataFrame: A DataFrame containing sediment flux per reach in cubic meters per year.
    """

    # Select the appropriate sediment flux combination
    if qs_inclusion == 1:
        sed_flux_df["Selected_Flux_kg_s"] = (
            sed_flux_df["mean_BedloadFlux_kg_s"] + sed_flux_df["mean_SuspendedBedFlux_kg_s"]
        )
    elif qs_inclusion == 2:
        sed_flux_df["Selected_Flux_kg_s"] = (
            sed_flux_df["mean_BedloadFlux_kg_s"] + 
            sed_flux_df["mean_SuspendedBedFlux_kg_s"] + 
            sed_flux_df["mean_WashloadFlux_kg_s"]
        )
    else:
        raise ValueError("qs_inclusion must be either 1 (Bedload + Suspended Bedload) or 2 (Bedload + Suspended Bedload + Washload).")

    # Convert mass flux (kg/s) to volumetric flux (m³/year)
    seconds_per_year = 60 * 60 * 24 * 365  # 31,536,000 seconds in a year
    sed_flux_df["SedimentFlux_m3_yr"] = (sed_flux_df["Selected_Flux_kg_s"] / sediment_bulk_density) * seconds_per_year
    
    # Ensure ds_order is an integer
    sed_flux_df["ds_order"] = sed_flux_df["ds_order"].astype(int)

    # Retain only necessary columns
    result_df = sed_flux_df[["ds_order", "SedimentFlux_m3_yr"]]

    return result_df

In [ ]:
beni_sedflux = calculate_sed_flux(beni)
print(beni_sedflux)

In [ ]:
def calculate_channel_slope(channel_gdf):
    """
    Calculates the slope of a river channel from its main channel line.
    
    Parameters:
        channel_gdf (gpd.GeoDataFrame): A GeoDataFrame containing the main channel line geometry.
    
    Returns:
        float: The channel slope (dimensionless).
    """
    
    # Extract the first (start) and last (end) points of the main channel line
    main_channel_line = channel_gdf.geometry.iloc[0]  # Assume the first row is the main channel
    start_point = main_channel_line.coords[0]  # First coordinate (lon, lat)
    end_point = main_channel_line.coords[-1]   # Last coordinate (lon, lat)

    # Convert to Earth Engine Points
    start_ee = ee.Geometry.Point(start_point)
    end_ee = ee.Geometry.Point(end_point)

    # Get elevation data from NASA SRTM (30m resolution)
    srtm = ee.Image("USGS/SRTMGL1_003")  # SRTM 30m DEM

    # Extract elevation at start and end points
    start_elev = srtm.sample(start_ee, 30).first().get("elevation").getInfo()
    end_elev = srtm.sample(end_ee, 30).first().get("elevation").getInfo()

    # Calculate elevation difference
    elevation_diff = start_elev - end_elev

    # Calculate geodesic length of the channel (meters)
    channel_length = channel_gdf.length.sum()
    print(channel_length)

    # Compute channel slope (dimensionless)
    slope = abs(elevation_diff / channel_length)

    return slope

In [ ]:
# Load the shapefile into a GeoDataFrame
channel_gdf = gpd.read_file(r"D:\Dissertation\Data\RiverMapping\Channels\Beni\reach_4\2024\main_channel.shp")

# Ensure it is a LineString geometry
if not all(channel_gdf.geometry.geom_type == "LineString"):
    raise ValueError("The shapefile must contain LineString geometries only.")

# Display the first few rows
print(channel_gdf.head())

In [ ]:
slope = calculate_channel_slope(channel_gdf)
print(slope)

In [ ]:
def calculate_channel_width(channel_mask, channel_gdf, pixel_resolution=30, raster_path=None):
    """
    Estimates the channel width by dividing the wetted area by the main channel length.

    Parameters:
        channel_mask (np.ndarray): A 2D array where 1 indicates water (channel) pixels and 0 is non-water.
        channel_gdf (gpd.GeoDataFrame): A GeoDataFrame containing the main channel line geometry.
        pixel_resolution (float, optional): The spatial resolution of the raster in meters (default = 30m).
        raster_path (str, optional): Path to the original raster file for reprojection.

    Returns:
        float: Estimated channel width in meters.
    """

    # --- 1️⃣ Check and Assign CRS if Missing ---
    if channel_gdf.crs is None:
        print("⚠️ Warning: No CRS found in channel shapefile. Assuming WGS84 (EPSG:4326).")
        channel_gdf.set_crs(epsg=4326, inplace=True)  # Assign WGS84 if missing

    # --- 2️⃣ Reproject to a Projected CRS (UTM) ---
    projected_crs = channel_gdf.estimate_utm_crs()  # Find the best UTM CRS
    channel_gdf = channel_gdf.to_crs(projected_crs)  # Convert to UTM for accurate distance

    # --- 3️⃣ Reproject Raster to Match CRS ---
    if raster_path:
        with rasterio.open(raster_path) as src:
            transform, width, height = calculate_default_transform(
                src.crs, projected_crs, src.width, src.height, *src.bounds
            )

            kwargs = src.meta.copy()
            kwargs.update({"crs": projected_crs, "transform": transform, "width": width, "height": height})

            # Reproject the raster
            with rasterio.open("reprojected_raster.tif", "w", **kwargs) as dst:
                for i in range(1, src.count + 1):
                    reproject(
                        source=rasterio.band(src, i),
                        destination=rasterio.band(dst, i),
                        src_transform=src.transform,
                        src_crs=src.crs,
                        dst_transform=transform,
                        dst_crs=projected_crs,
                        resampling=Resampling.nearest,
                    )

            # Load the reprojected raster
            with rasterio.open("reprojected_raster.tif") as reprojected_src:
                channel_mask = reprojected_src.read(1)  # Read first band
                pixel_resolution = reprojected_src.res[0]  # Update pixel resolution after reprojection

    # --- 4️⃣ Compute Wetted Area Using Projected Pixel Resolution ---
    water_pixel_count = np.sum(channel_mask == 1)  # Count water pixels
    wetted_area = water_pixel_count * (pixel_resolution ** 2)  # Convert to square meters
    print(wetted_area)

    # --- 5️⃣ Compute Total Channel Length ---
    total_channel_length = channel_gdf.length.sum()  # Sum all segment lengths in meters
    print(total_channel_length)

    # --- 6️⃣ Compute Estimated Channel Width ---
    estimated_width = wetted_area / total_channel_length

    return estimated_width


In [ ]:
with rasterio.open(r"D:\Dissertation\Data\RiverMapping\RiverMasks\Beni\reach_4\Cleaned\Beni_reach_4_2024_DSWE_level_3_cleaned.tif") as src:
    channel_mask = src.read(1)  # Read first band
    pixel_size = src.res[0]  # Assuming square pixels, take resolution

# Load main channel shapefile
channel_gdf = gpd.read_file(r"D:\Dissertation\Data\RiverMapping\Channels\Beni\reach_4\2024\main_channel.shp")

# Compute estimated width
channel_width = calculate_channel_width(channel_mask, channel_gdf, pixel_size)
print(f"Estimated Channel Width: {channel_width:.2f} meters")

## Run Monte Carlo simulation to calculate total transit time from the number of storage events and storage time distributions

In [13]:
def monte_carlo_reach_transit_time(tfp_df: pd.DataFrame, storage_df: pd.DataFrame, reach_number: int, num_iterations: int = 10_000) -> pd.DataFrame:
    """
    Runs a Monte Carlo Simulation to compute reach transit time values.
    
    Parameters:
        tfp_df (pd.DataFrame): DataFrame containing TFP values for a specific reach.
        storage_df (pd.DataFrame): DataFrame containing storage events data for multiple reaches.
        reach_number (int): The reach number for which to calculate transit time.
        num_iterations (int): Number of iterations for the Monte Carlo simulation (default is 10,000).

    Returns:
        pd.DataFrame: DataFrame containing the reach transit time distribution.
    """
    # Ensure column names are properly formatted
    storage_df.columns = storage_df.columns.str.strip()
    
    reach_storage_events = storage_df.loc[storage_df["Reach"] == reach_number, "number_storage_events"].values
    if len(reach_storage_events) == 0:
        raise ValueError(f"No storage event data found for Reach {reach_number}.")
    reach_storage_events = reach_storage_events[0]
    
    # Randomly sample TFP values from the distribution
    random_tfp_samples = np.random.choice(tfp_df["TCB"], size=num_iterations, replace=True)
    
    # Compute reach transit time
    reach_transit_time = random_tfp_samples * reach_storage_events
    
    # Create DataFrame for results
    return pd.DataFrame({"reach_transit_time_yr": reach_transit_time})

def process_all_reaches(work_dir: str, output_subfolder: str = "RTT_fromTCB_Distributions"):
    """
    Processes all reach transit time distributions in the working directory.
    
    Parameters:
        work_dir (str): Path to the working directory containing relevant data files.
        output_subfolder (str): Name of the subfolder where output CSVs will be saved.
    """
    # Define input file paths
    storage_events_path = r"C:\Users\huckr\Desktop\Scholarships_Apps\FINESST\Figures\FINESST_storage_transit_calcs.xlsx"
    tfp_files = [f for f in os.listdir(work_dir) if f.startswith("Reach_") and f.endswith("_TCB_distribution.csv")]
    
    # Load storage events data
    xls = pd.ExcelFile(storage_events_path)
    beni_df = pd.read_excel(xls, sheet_name="Bermejo")
    
    # Create output directory
    output_dir = os.path.join(work_dir, output_subfolder)
    os.makedirs(output_dir, exist_ok=True)
    
    # Process each reach
    for tfp_file in tfp_files:
        reach_number = int(tfp_file.split("_")[1])  # Extract reach number from filename
        tfp_df = pd.read_csv(os.path.join(work_dir, tfp_file))
        
        # Run Monte Carlo Simulation
        reach_transit_time_df = monte_carlo_reach_transit_time(tfp_df, beni_df, reach_number)
        
        # Save output
        output_path = os.path.join(output_dir, f"Reach_{reach_number}_RTT_fromTCB_distribution.csv")
        reach_transit_time_df.to_csv(output_path, index=False)
        
        print(f"Saved: {output_path}")


In [14]:
wd = r'D:\Dissertation\Data\RiverMapping\Mobility\Bermejo\TCB_Distributions'
process_all_reaches(wd, 'RTT_fromTCB_Distributions')

Saved: D:\Dissertation\Data\RiverMapping\Mobility\Bermejo\TCB_Distributions\RTT_fromTCB_Distributions\Reach_1_RTT_fromTCB_distribution.csv
Saved: D:\Dissertation\Data\RiverMapping\Mobility\Bermejo\TCB_Distributions\RTT_fromTCB_Distributions\Reach_2_RTT_fromTCB_distribution.csv
Saved: D:\Dissertation\Data\RiverMapping\Mobility\Bermejo\TCB_Distributions\RTT_fromTCB_Distributions\Reach_3_RTT_fromTCB_distribution.csv
Saved: D:\Dissertation\Data\RiverMapping\Mobility\Bermejo\TCB_Distributions\RTT_fromTCB_Distributions\Reach_4_RTT_fromTCB_distribution.csv
Saved: D:\Dissertation\Data\RiverMapping\Mobility\Bermejo\TCB_Distributions\RTT_fromTCB_Distributions\Reach_5_RTT_fromTCB_distribution.csv
Saved: D:\Dissertation\Data\RiverMapping\Mobility\Bermejo\TCB_Distributions\RTT_fromTCB_Distributions\Reach_6_RTT_fromTCB_distribution.csv
Saved: D:\Dissertation\Data\RiverMapping\Mobility\Bermejo\TCB_Distributions\RTT_fromTCB_Distributions\Reach_7_RTT_fromTCB_distribution.csv
Saved: D:\Dissertation\Data

## Calculate distributions for total alluvial transit time

In [22]:
def monte_carlo_total_transit_time(rtt_dir: str, river_name: str, num_iterations: int = 10_000):
    """
    Runs a Monte Carlo simulation to compute the total river transit time distribution.
    
    Parameters:
        rtt_dir (str): Directory containing reach transit time distributions.
        river_name (str): Name of the river for output file naming.
        num_iterations (int): Number of iterations for the Monte Carlo simulation (default is 10,000).

    Returns:
        pd.DataFrame: DataFrame containing the total river transit time distribution.
    """
    rtt_files = [f for f in os.listdir(rtt_dir) if f.startswith("Reach_") and f.endswith("_RTT_fromTCB_distribution.csv")]
    reach_dfs = [pd.read_csv(os.path.join(rtt_dir, file)) for file in rtt_files]
    
    # Ensure all distributions have the correct format
    for df in reach_dfs:
        if "reach_transit_time_yr" not in df.columns:
            raise KeyError("Expected column 'reach_transit_time_yr' missing from a reach distribution file.")
    
    # Monte Carlo simulation: summing random samples from each reach distribution
    total_transit_times = [sum(np.random.choice(df["reach_transit_time_yr"], 1)[0] for df in reach_dfs) for _ in range(num_iterations)]
    
    # Compute statistics
    stats = {
        "Mean": np.mean(total_transit_times),
        "Standard Deviation": np.std(total_transit_times),
        "Min": np.min(total_transit_times),
        "1st Quartile": np.percentile(total_transit_times, 25),
        "Median": np.median(total_transit_times),
        "3rd Quartile": np.percentile(total_transit_times, 75),
        "Max": np.max(total_transit_times)
    }
    
    for stat, value in stats.items():
        print(f"{stat}: {value}")
    
    # Create DataFrame for results
    total_transit_time_df = pd.DataFrame({"total_transit_time_yr": total_transit_times})
    
    # Save output
    output_path = os.path.join(rtt_dir, f"{river_name}_TTT_fromTCB_distribution.csv")
    total_transit_time_df.to_csv(output_path, index=False)
    
    print(f"Saved: {output_path}")
    return total_transit_time_df


In [26]:
monte_carlo_total_transit_time(r'D:\Dissertation\Data\RiverMapping\Mobility\Bermejo\RTT_fromTCB_Distributions', 'Bermejo', 10000)

Mean: 13566.868321286403
Standard Deviation: 1701.077686646581
Min: 10976.456775682582
1st Quartile: 12619.830372423265
Median: 13029.438888396508
3rd Quartile: 13752.182449898666
Max: 25563.612976883112
Saved: D:\Dissertation\Data\RiverMapping\Mobility\Bermejo\RTT_fromTCB_Distributions\Bermejo_TTT_fromTCB_distribution.csv


,total_transit_time_yr
0,13070.253949
1,12767.528591
2,12785.305558
3,14096.059628
4,13934.345422
...,...
9995,12419.390133
9996,13496.188568
9997,12511.151890
9998,13008.388194


In [27]:
def calculate_ttt_statistics(ttt_csv_path: str):
    """
    Reads a total transit time distribution CSV and prints key statistics.
    
    Parameters:
        ttt_csv_path (str): Path to the total transit time distribution CSV file.
    """
    ttt_df = pd.read_csv(ttt_csv_path)
    
    if "total_transit_time_yr" not in ttt_df.columns:
        raise KeyError("Expected column 'total_transit_time_yr' missing from the input file.")
    
    # Compute statistics
    stats = {
        "Mean": np.mean(ttt_df["total_transit_time_yr"]),
        "Standard Deviation": np.std(ttt_df["total_transit_time_yr"]),
        "Min": np.min(ttt_df["total_transit_time_yr"]),
        "1st Quartile": np.percentile(ttt_df["total_transit_time_yr"], 25),
        "Median": np.median(ttt_df["total_transit_time_yr"]),
        "3rd Quartile": np.percentile(ttt_df["total_transit_time_yr"], 75),
        "Max": np.max(ttt_df["total_transit_time_yr"])
    }
    
    for stat, value in stats.items():
        print(f"{stat}: {value}")
    
    return stats

In [37]:
calculate_ttt_statistics(r"D:\Dissertation\Data\RiverMapping\Mobility\Beni\Beni_TTT_fromTCB_distribution.csv")

Mean: 5403.3538147015415
Standard Deviation: 141.08441148352148
Min: 4877.258722602724
1st Quartile: 5307.332000266164
Median: 5403.638093746336
3rd Quartile: 5501.716028022813
Max: 5930.325937854683


{'Mean': 5403.3538147015415,
 'Standard Deviation': 141.08441148352148,
 'Min': 4877.258722602724,
 '1st Quartile': 5307.332000266164,
 'Median': 5403.638093746336,
 '3rd Quartile': 5501.716028022813,
 'Max': 5930.325937854683}